# Aggregating large files with pandas

In [23]:
import pandas as pd
from dfply import *

## Uber data

For this exercise, we will be using the [FiveThirtyEight Uber TLC FOIL data](), be sure that you have download and installed these data.

**Note:** Make sure that you have unzipped the file `./data/uber/uber-trip-data/uber-raw-data-janjune-15.csv.zip`, which should result in the file with the following locaiton: `./data/uber/uber-trip-data/uber-raw-data-janjune-15.csv`.

In [24]:
!git clone https://github.com/fivethirtyeight/uber-tlc-foil-response.git ./data/uber

fatal: destination path './data/uber' already exists and is not an empty directory.


## New Python files

We will be using the following two new python packages.  Insure that you have them both installed and remember to restart the notebook after any installation.

In [25]:
!pip install unpythonic
!pip install macropy3

You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Working with large files

When working with large files,

* Use `chunksize` to get a data frame iterator
    * `chunksize` is the number of rows
`chunksize` is the number of rows
* Iterators are lazy $\rightarrow$ one chunk at a time
* Iterators give you one pass at the data

## General Workflow

* Read chunk $\rightarrow$ process chunk $\rightarrow$ write/aggregate chunk $\rightarrow$ more to next chunk
* Last chunk is lost when you read the next chunk

## Safe operations on big data

* **Aggregate** to a manageable size
* **Filter** to a manageable size
* **Read+Write** Read chuck $\rightarrow$ Process chuck $\rightarrow$ Write chunk
* Use **generator expressions**
    * like list comprehensions, but with `()`
    * More about this at the end of the course

## Big data best practice

* Use `toolz.first` to grab the first chunk
* Prototype on a small chunk of data
* Run your code on all chunks later


## Example 1 - Counting weekday uber pickup frequency

Workflow:
* Create a dataframe iterator
* Prototype by reading/processing the first chunk
    * Get names/types correct
    * Aggregate chunk
* Reset iterator and process all chunks
* Use the accumulator pattern to combine chunk aggregates

## Step 1 - Prototyping your code

* Now build/test your code on a small chunk
* Good general coding technique

#### Create a data frame iterator

In [26]:
c_size = 10000
df_iter = pd.read_csv("./data/uber/uber-trip-data/uber-raw-data-apr14.csv",
                      chunksize=c_size)
df_iter

## What is `df_iter`?

* Using `chunksize` $\rightarrow$ dataframe iterator
* Lazily returns chunks of data
* Won't fill up memory

#### Read and prototype on the first chunk

In [27]:
from toolz import first 

first_chunk = first(df_iter)
first_chunk.shape

(10000, 4)

In [28]:
first_chunk.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


#### Making a list of clean names

In [29]:
first_chunk.columns

Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')

In [30]:
new_names = ['date', 'lat', 'lon', 'base']

#### Inspecting and fixing types

In [31]:
first_chunk.dtypes

Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object

In [32]:
date_cols = ['date']

#### Test by re-reading the data with new parameters

In [33]:
c_size = 10000
date_cols = ['date']
new_names = ['date', 'lat', 'lon', 'base']
df_iter = pd.read_csv("./data/uber-raw-data-apr14.csv", 
                      header=0, names=new_names, # This is now we fix names at read time
                      parse_dates=date_cols,
                      chunksize=c_size)
first_chunk = first(df_iter)
first_chunk.dtypes

date    datetime64[ns]
lat            float64
lon            float64
base            object
dtype: object

In [34]:
first_chunk.head()

,date,lat,lon,base
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512


## Step 2 - Computing the weekday distribution for each chunk

Let's count how many pickups occured on each weekday.  To do this we will

1. Prototype the code on the first chunk;
2. Create helper functions
3. Aggregate all chunks in a comprehension

## What we need to accumplish

1. Add the weekday names
2. Count the frequency for each weekday

#### Prototyping on the first chunk

In [35]:
first_chunk.head()

,date,lat,lon,base
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512


#### Expression for adding the weekday name to the data frame

In [36]:
from calendar import day_name
weekday_categories = pd.CategoricalDtype(categories=list(day_name),
                                     ordered=True)
(first_chunk 
 >> mutate(weekday = X.date.dt.weekday_name.astype(weekday_categories)) 
 >> head(2))

,date,lat,lon,base,weekday
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,Tuesday
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,Tuesday


#### Encapsulate the expression in a helper function and test on first chunk

In [37]:
from functoolz import pipeable
add_weekday = pipeable(lambda df: (df 
                                   >> mutate(weekday = X.date.dt.weekday_name.astype(weekday_categories)) 
                                   >> select(X.date, X.weekday)))
first_chunk >> add_weekday >> head(3)

,date,weekday
0,2014-04-01 00:11:00,Tuesday
1,2014-04-01 00:17:00,Tuesday
2,2014-04-01 00:21:00,Tuesday


#### Expression for chunk aggregation

In [38]:
(first_chunk 
 >> add_weekday 
 >> group_by(X.weekday) 
 >> summarise(cnt = n(X.weekday)))

,weekday,cnt
0,Monday,1376
1,Tuesday,1850
2,Wednesday,1336
3,Thursday,1482
4,Friday,1827
5,Saturday,1309
6,Sunday,820


#### Functions for chunk aggregation

In [39]:
daily_cnts = pipeable(lambda df: (df 
                                  >> group_by(X.weekday) 
                                  >> summarise(cnt = n(X.weekday))))
first_chunk >> add_weekday >> daily_cnts

,weekday,cnt
0,Monday,1376
1,Tuesday,1850
2,Wednesday,1336
3,Thursday,1482
4,Friday,1827
5,Saturday,1309
6,Sunday,820


## Step 3 - Create list of chunk counts

Note that `chunk >> add_weekday >> daily_cnts` 

* Reduces each chunk to a managable size
* It is safe to save all the results

#### Reset  `df_iter`

In [40]:
c_size = 40000
df_iter = pd.read_csv("./data/uber/uber-trip-data/uber-raw-data-apr14.csv",
                      header=0, names=new_names, 
                      parse_dates=date_cols,
                      chunksize=c_size,
                      sep = ',',
                      engine = 'python')

#### Watching the paint dry

Without a progress bar, this can be unbearable.

In [41]:
#chunk_cnts = [chunk >> add_weekday >> daily_cnts
              #for i, chunk in enumerate(df_iter)]

In [79]:
c_size = 40000
df_iter = pd.read_csv("./data/uber/uber-trip-data/uber-raw-data-apr14.csv",
                      header=0, names=new_names, 
                      parse_dates=date_cols,
                      chunksize=c_size,
                      sep = ',',
                      engine = 'python')

In [43]:
from unpythonic import begin
chunk_cnts = [begin(print("processing chunk {0}, aggregating {1} rows".format(i, len(chunk))), 
                    chunk >> add_weekday >> daily_cnts)
              for i, chunk in enumerate(df_iter)]

processing chunk 0, aggregating 40000 rows
processing chunk 1, aggregating 40000 rows
processing chunk 2, aggregating 40000 rows
processing chunk 3, aggregating 40000 rows
processing chunk 4, aggregating 40000 rows
processing chunk 5, aggregating 40000 rows
processing chunk 6, aggregating 40000 rows
processing chunk 7, aggregating 40000 rows
processing chunk 8, aggregating 40000 rows
processing chunk 9, aggregating 40000 rows
processing chunk 10, aggregating 40000 rows
processing chunk 11, aggregating 40000 rows
processing chunk 12, aggregating 40000 rows
processing chunk 13, aggregating 40000 rows
processing chunk 14, aggregating 4516 rows


In [44]:
chunk_cnts[2]

,weekday,cnt
0,Monday,3889
1,Tuesday,6327
2,Wednesday,6442
3,Thursday,7418
4,Friday,6213
5,Saturday,5908
6,Sunday,3803


#### Refactored

In [45]:
from unpythonic import begin
print_progress = pipeable(lambda i, chunk: begin(print("processing chunk {0}, aggregating {1} rows".format(i, len(chunk))), chunk))
chunk_cnts = [begin(print_progress(i, chunk),
                    chunk >> add_weekday >> daily_cnts)
              for i, chunk in enumerate(df_iter)]


ValueError: I/O operation on closed file.

## Using `begin` to add print statements in an expression

* The expression `begin(print('hi'), expr)` will
    * print hi
    * return the value of expr
* A `begin` expression is the same as the last argument
    * Except print/side-effects
* Nice way to add a progress marker

In [46]:
begin(print('hi'), 2)

hi


2

## Step 3 - Combine the counts with the accumulator pattern

#### Figure out how to combine too chunk counts

In [47]:
cnt1 = chunk_cnts[0]
cnt2 = chunk_cnts[1]

In [48]:
cnt1

,weekday,cnt
0,Monday,3945
1,Tuesday,9609
2,Wednesday,7670
3,Thursday,5493
4,Friday,5552
5,Saturday,4665
6,Sunday,3066


In [49]:
cnt2

,weekday,cnt
0,Monday,5336
1,Tuesday,4781
2,Wednesday,7897
3,Thursday,5499
4,Friday,7151
5,Saturday,5508
6,Sunday,3828


## Combining chunk aggregations

To combine the chunk aggregation

1. `pd.concat` $\rightarrow$ stack the columns
2. Perform another aggregation to combine

#### Concatenating the chunk aggregations

In [50]:
pd.concat([cnt1, cnt2], sort = False) 

,weekday,cnt
0,Monday,3945
1,Tuesday,9609
2,Wednesday,7670
3,Thursday,5493
4,Friday,5552
5,Saturday,4665
6,Sunday,3066
0,Monday,5336
1,Tuesday,4781
2,Wednesday,7897


#### Aggregate to combine counts

In [51]:
(pd.concat([cnt1, cnt2], sort = False) 
 >> group_by(X.weekday) 
 >> summarise(cnt = X.cnt.sum()))

,weekday,cnt
0,Monday,9281
1,Tuesday,14390
2,Wednesday,15567
3,Thursday,10992
4,Friday,12703
5,Saturday,10173
6,Sunday,6894


#### Make a combine helper function

In [52]:
combine = lambda df1, df2: (pd.concat([df1, df2], sort = False) 
                            >> group_by(X.weekday) 
                            >> summarise(cnt = X.cnt.sum()))
combine(cnt1, cnt2)

,weekday,cnt
0,Monday,9281
1,Tuesday,14390
2,Wednesday,15567
3,Thursday,10992
4,Friday,12703
5,Saturday,10173
6,Sunday,6894


## Key-value aggregation

* Example of key-value aggregation
    * **Data:** Key-value pairs
    * **Operation:** group + aggregate
* `spark.RDD` consists of key-value pairs
    * Inspired by `MapReduce`
    * Foundation of all of `spark`/`pyspark`
* Key feature: sums of counts are counts
    * Only works for some aggregations
    * Careful how you combine 

## Key-value Aggregation

<img src="./img/key_value_agg.gif" width=1000>

#### Initialize the accumulator

In [53]:
cnts = chunk_cnts[0]
cnts

,weekday,cnt
0,Monday,3945
1,Tuesday,9609
2,Wednesday,7670
3,Thursday,5493
4,Friday,5552
5,Saturday,4665
6,Sunday,3066


#### Process the remaining chunks

In [54]:
for chunk_cnt in chunk_cnts[1:]:
    cnts = combine(cnts, chunk_cnt)
cnts

,weekday,cnt
0,Monday,60861
1,Tuesday,91185
2,Wednesday,108631
3,Thursday,85067
4,Friday,90303
5,Saturday,77218
6,Sunday,51251


## Visualizing the results

In [65]:
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

ax = sns.catplot(x="weekday", y = 'cnt',kind="bar", palette="ch:.25", data=cnts)

AttributeError: module 'seaborn' has no attribute 'catplot'

## Fixing the labels

In [59]:
import matplotlib.pylab as plt
ax = (sns
      .catplot(x="weekday", 
               y = 'cnt',
               kind="bar", 
               palette="ch:.25", 
               data=cnts))

ax.set_xticklabels(ax.ax.get_xticklabels(), rotation=40, ha="right")
ax.set(title='Uber Pick-Ups by Day of the Week',
       xlabel='Day of the Week', 
       ylabel='Num of Pick-ups')
plt.tight_layout()
plt.show()

AttributeError: module 'seaborn' has no attribute 'catplot'

SyntaxError: invalid syntax (<ipython-input-89-6e25dc9ac254>, line 1)

## <font color="red"> Exercise 2 </font>

Compute and visualize the hourly frequencies for Uber pick-ups in April 2014.

In [73]:
from functoolz import pipeable
add_hour = pipeable(lambda df:(df
                               >> mutate(hour = X.date.dt.hour) 
                               >> select(X.date, X.hour)))
first_chunk >> add_hour >> head(3)

,date,hour
0,2014-04-01 00:11:00,0
1,2014-04-01 00:17:00,0
2,2014-04-01 00:21:00,0


In [75]:
daily_cnts = pipeable(lambda df:(df 
                                 >> group_by(X.hour) 
                                 >> summarise(cnt = n(X.hour))))
first_chunk >> add_hour >> daily_cnts

,hour,cnt
0,0,173
1,1,107
2,2,64
3,3,55
4,4,82
5,5,151
6,6,358
7,7,467
8,8,390
9,9,306


In [80]:
from unpythonic import begin
print_progress = pipeable(lambda i, chunk: begin(print("processing chunk {0}, aggregating {1} rows".format(i, len(chunk))), chunk))
chunk_cnts = [begin(print_progress(i, chunk),
                    chunk >> add_hour >> daily_cnts)
              for i, chunk in enumerate(df_iter)]

processing chunk 0, aggregating 40000 rows
processing chunk 1, aggregating 40000 rows
processing chunk 2, aggregating 40000 rows
processing chunk 3, aggregating 40000 rows
processing chunk 4, aggregating 40000 rows
processing chunk 5, aggregating 40000 rows
processing chunk 6, aggregating 40000 rows
processing chunk 7, aggregating 40000 rows
processing chunk 8, aggregating 40000 rows
processing chunk 9, aggregating 40000 rows
processing chunk 10, aggregating 40000 rows
processing chunk 11, aggregating 40000 rows
processing chunk 12, aggregating 40000 rows
processing chunk 13, aggregating 40000 rows
processing chunk 14, aggregating 4516 rows


In [83]:
combine = lambda df1, df2: (pd.concat([df1, df2], sort = False) 
                            >> group_by(X.hour) 
                            >> summarise(cnt = X.cnt.sum()))

In [84]:
for chunk_cnt in chunk_cnts[1:]:
    cnts = combine(cnts, chunk_cnt)
cnts

,hour,cnt
0,0.0,11245
1,1.0,7377
2,2.0,4678
3,3.0,4768
4,4.0,5690
5,5.0,8814
6,6.0,16973
7,7.0,22906
8,8.0,21374
9,9.0,16635


## Up Next

In [Lecture 3.3](./3_3_reading_processing_and_writing_large_files.ipynb), we will look an a second option for working with large files, which involves reading, processing, then writing each chunk to a new file.